In [1]:
import numpy as np
import astropy.constants as const
import matplotlib.pyplot as plt
from RMtools_1D import do_RMsynth_1D, do_RMclean_1D
from scipy import signal
import numba as nb
from numba_progress import ProgressBar

In [2]:
def ddf(x,sig):
    val = np.zeros_like(x)
    for i,p in enumerate(x):
        if p <= sig and x[i+1] > sig:
            val[i] = 1
    return val
def tophat(x, x1, x2):
    val = np.zeros_like(x)
    idx = (x >= x1) & (x<x2)
    val[idx] = 1
    return val
def screen(lsq, RM, psi, phi=None):
    P = 0.7*np.exp(2j*(np.deg2rad(psi)+RM*lsq))
    ret = {
        'Q': np.real(P),
        'U': np.imag(P),
        'PI': np.abs(P),
        'PA': np.rad2deg(0.5*np.arctan2(np.imag(P),np.real(P))) % 180
    }
    ret = P
    if phi is not None:
        inv = ddf(phi, RM)
        ret = (ret, inv)
    return ret
def double_screen(lsq, RM1, RM2, psi1, psi2, phi=None):
    P1 = 0.7*np.exp(2j*(np.deg2rad(psi1)+RM1*lsq))
    P2 = 0.3*np.exp(2j*(np.deg2rad(psi2)+RM2*lsq))
    P = P1 + P2
    ret = {
        'Q': np.real(P),
        'U': np.imag(P),
        'PI': np.abs(P),
        'PA': np.rad2deg(0.5*np.arctan2(np.imag(P),np.real(P))) % 180
    }
    ret = P
    if phi is not None:
        inv1 = ddf(phi, RM1)
        inv2 = ddf(phi, RM2)
        inv = inv1 + inv2
        ret = (ret, inv)
    return ret
def slab(lsq, RM, psi, phi=None):
    P = 0.7*np.exp(2j*(np.deg2rad(psi)+0.5*RM*lsq))*np.sin(RM*lsq)/(RM*lsq)
    ret = {
        'Q': np.real(P),
        'U': np.imag(P),
        'PI': np.abs(P),
        'PA': np.rad2deg(0.5*np.arctan2(np.imag(P),np.real(P))) % 180
    }
    ret = P
    if phi is not None:
        inv = tophat(phi, 0, RM)
        ret = (ret, inv)
    return ret

In [3]:
rm = np.pi

start = 744
stop = 1032
step = 1
f_racs = np.arange(start, stop, step)*1e6
lsq_racs = (2.998e8/f_racs)**2

p_racs = screen(lsq_racs, rm, 10)
data = [
    f_racs,
    p_racs.real,
    p_racs.imag,
    np.ones_like(p_racs.real)*0.001,
    np.ones_like(p_racs.imag)*0.001,
]


In [5]:
_ = do_RMsynth_1D.run_rmsynth(
    data,
    verbose=True,
    nSamples=100,   
)

> Trying [freq_Hz, I, Q, U, dI, dQ, dU] ...failed.
> Trying [freq_Hz, q, u,  dq, du] ... success.
Successfully read in the Stokes spectra.
Warn: no Stokes I data in use.
Plotting the input data and spectral index fit.
PhiArr = -3976.32 to 3976.32 by 0.45 (17865 chans).
Weight type is 'variance'.
Running RM-synthesis by channel.
  [========================================] 100%
Calculating 1D RMSF and replicating along X & Y axes.
> RM-synthesis completed in 14.70 seconds.

--------------------------------------------------------------------------------
RESULTS:

FWHM RMSF = 48.83 rad/m^2
Pol Angle = 31.1 (+/-0.002412) deg
Pol Angle 0 = 10 (+/-0.01295) deg
Peak FD = 3.142 (+/-0.002055) rad/m^2
freq0_GHz = 0.8757 
I freq0 = 1 Jy/beam
Peak PI = 0.7 (+/-5.893e-05) Jy/beam
QU Noise = 0.001 Jy/beam
FDF Noise (theory)   = 5.893e-05 Jy/beam
FDF Noise (Corrected MAD) = 0.002506 Jy/beam
FDF Noise (rms)   = 0.01283 Jy/beam
FDF SNR = 1.188e+04 
sigma_add(q) = 0.00746 (+0.02796, -0.005457)
sigma_ad

In [6]:
from RMutils.util_misc import progress


@nb.njit(parallel=True, fastmath=True,)
def _rmsynth(FDFcube, phiArr_radm2, nPhi, KArr, pCube, a, progress):
    for i in nb.prange(nPhi):
        arg = np.expand_dims(np.expand_dims(np.exp(-2.0j * phiArr_radm2[i] * a), axis=-1), axis=-1)
        FDFcube[i,:,:] =  KArr * np.sum(pCube * arg, axis=0)
        progress.update(1)
    return FDFcube
def do_rmsynth_planes_fast(dataQ, dataU, lambdaSqArr_m2, phiArr_radm2, 
                      weightArr=None, lam0Sq_m2=None, nBits=32, verbose=False,
                      log=print):
    """Perform RM-synthesis on Stokes Q and U cubes (1,2 or 3D). This version
    of the routine loops through spectral planes and is faster than the pixel-
    by-pixel code. This version also correctly deals with isolated clumps of
    NaN-flagged voxels within the data-cube (unlikely in interferometric cubes,
    but possible in single-dish cubes). Input data must be in standard python
    [z,y,x] order, where z is the frequency axis in ascending order.

    dataQ           ... 1, 2 or 3D Stokes Q data array
    dataU           ... 1, 2 or 3D Stokes U data array
    lambdaSqArr_m2  ... vector of wavelength^2 values (assending freq order)
    phiArr_radm2    ... vector of trial Faraday depth values
    weightArr       ... vector of weights, default [None] is Uniform (all 1s)
    nBits           ... precision of data arrays [32]
    verbose         ... print feedback during calculation [False]
    log             ... function to be used to output messages [print]
    
    """
    
    # Default data types
    dtFloat = "float" + str(nBits)
    dtComplex = "complex" + str(2*nBits)

    # Set the weight array
    if weightArr is None:
        weightArr = np.ones(lambdaSqArr_m2.shape, dtype=dtFloat)
    weightArr = np.where(np.isnan(weightArr), 0.0, weightArr)
    
    # Sanity check on array sizes
    if not weightArr.shape  == lambdaSqArr_m2.shape:
        log("Err: Lambda^2 and weight arrays must be the same shape.")
        return None, None
    if not dataQ.shape == dataU.shape:
        log("Err: Stokes Q and U data arrays must be the same shape.")
        return None, None
    nDims = len(dataQ.shape)
    if not nDims <= 3:
        log("Err: data dimensions must be <= 3.")
        return None, None
    if not dataQ.shape[0] == lambdaSqArr_m2.shape[0]:
        log("Err: Data depth does not match lambda^2 vector ({} vs {}).".format(dataQ.shape[0], lambdaSqArr_m2.shape[0]), end=' ')
        log("     Check that data is in [z, y, x] order.")
        return None, None
    
    # Reshape the data arrays to 3 dimensions
    if nDims==1:
        dataQ = np.reshape(dataQ, (dataQ.shape[0], 1, 1))
        dataU = np.reshape(dataU, (dataU.shape[0], 1, 1))
    elif nDims==2:
        dataQ = np.reshape(dataQ, (dataQ.shape[0], dataQ.shape[1], 1))
        dataU = np.reshape(dataU, (dataU.shape[0], dataU.shape[1], 1))
    
    # Create a complex polarised cube, B&dB Eqns. (8) and (14)
    # Array has dimensions [nFreq, nY, nX]
    pCube = (dataQ + 1j * dataU) * weightArr[:, np.newaxis, np.newaxis]
    
    # Check for NaNs (flagged data) in the cube & set to zero
    mskCube = np.isnan(pCube)
    pCube = np.nan_to_num(pCube)
    
    # If full planes are flagged then set corresponding weights to zero
    mskPlanes =  np.sum(np.sum(~mskCube, axis=1), axis=1)
    mskPlanes = np.where(mskPlanes==0, 0, 1)
    weightArr *= mskPlanes
    
    # Initialise the complex Faraday Dispersion Function cube
    nX = dataQ.shape[-1]
    nY = dataQ.shape[-2]
    nPhi = phiArr_radm2.shape[0]
    FDFcube = np.zeros((nPhi, nY, nX), dtype=dtComplex)

    # lam0Sq_m2 is the weighted mean of lambda^2 distribution (B&dB Eqn. 32)
    # Calculate a global lam0Sq_m2 value, ignoring isolated flagged voxels
    K = 1.0 / np.sum(weightArr)
    if lam0Sq_m2 is None:
        lam0Sq_m2 = K * np.sum(weightArr * lambdaSqArr_m2)
    if not np.isfinite(lam0Sq_m2): #Can happen if all channels are NaNs/zeros
        lam0Sq_m2=0.
    
    # The K value used to scale each FDF spectrum must take into account
    # flagged voxels data in the datacube and can be position dependent
    weightCube =  np.invert(mskCube) * weightArr[:, np.newaxis, np.newaxis]
    with np.errstate(divide='ignore', invalid='ignore'):
        KArr = np.true_divide(1.0, np.sum(weightCube, axis=0))
        KArr[KArr == np.inf] = 0
        KArr = np.nan_to_num(KArr)
        
    # Do the RM-synthesis on each plane
    a = lambdaSqArr_m2 - lam0Sq_m2
    with ProgressBar(total=nPhi, desc="Running RM-synthesis by channel", disable=not verbose) as progress:
        FDFcube = _rmsynth(FDFcube, phiArr_radm2, nPhi, KArr, pCube, a, progress)
    # Remove redundant dimensions in the FDF array
    FDFcube = np.squeeze(FDFcube)

    return FDFcube, lam0Sq_m2

In [7]:
from RMutils.util_RM import do_rmsynth_planes

In [8]:
phis = np.linspace(-1000, 1000, 10000)

In [21]:
# %%timeit
fdf, _ = do_rmsynth_planes(
    dataQ=p_racs.real,
    dataU=p_racs.imag,
    lambdaSqArr_m2=lsq_racs,
    phiArr_radm2=phis,
    weightArr=None,
    verbose=False
)

In [22]:
# %%timeit
fdf_fast, _ = do_rmsynth_planes_fast(
    dataQ=p_racs.real,
    dataU=p_racs.imag,
    lambdaSqArr_m2=lsq_racs,
    phiArr_radm2=phis,
    weightArr=None,
    verbose=False
)

In [23]:
np.array_equal(fdf, fdf_fast)

True

In [15]:
from RMutils.util_RM import extrap, get_rmsf_planes, fit_rmsf

@nb.njit
def _calc_rmsf(nPhi, phi2Arr, RMSFcube, KArr, weightCube, a):
    for i in nb.prange(nPhi):
        arg = np.expand_dims(np.expand_dims(np.exp(-2.0j * phi2Arr[i] * a), axis=-1), axis=-1)
        RMSFcube[i, :, :] = KArr * np.sum(weightCube * arg, axis=0)
    return RMSFcube

def get_rmsf_planes_fast(lambdaSqArr_m2, phiArr_radm2, weightArr=None, mskArr=None, 
                    lam0Sq_m2= None, double=True, fitRMSF=False,
                    fitRMSFreal=False, nBits=32, verbose=False,
                    log=print):
    """Calculate the Rotation Measure Spread Function from inputs. This version
    returns a cube (1, 2 or 3D) of RMSF spectra based on the shape of a
    boolean mask array, where flagged data are True and unflagged data False.
    If only whole planes (wavelength channels) are flagged then the RMSF is the
    same for all pixels and the calculation is done once and replicated to the
    dimensions of the mask. If some isolated voxels are flagged then the RMSF
    is calculated by looping through each wavelength plane, which can take some
    time. By default the routine returns the analytical width of the RMSF main
    lobe but can also use MPFIT to fit a Gaussian.
    
    lambdaSqArr_m2  ... vector of wavelength^2 values (assending freq order)
    phiArr_radm2    ... vector of trial Faraday depth values
    weightArr       ... vector of weights, default [None] is no weighting    
    maskArr         ... cube of mask values used to shape return cube [None]
    lam0Sq_m2       ... force a reference lambda^2 value (def=calculate) [None]
    double          ... pad the Faraday depth to double-size [True]
    fitRMSF         ... fit the main lobe of the RMSF with a Gaussian [False]
    fitRMSFreal     ... fit RMSF.real, rather than abs(RMSF) [False]
    nBits           ... precision of data arrays [32]
    verbose         ... print feedback during calculation [False]
    log             ... function to be used to output messages [print]

    """
    
    # Default data types
    dtFloat = "float" + str(nBits)
    dtComplex = "complex" + str(2*nBits)
    
    # For cleaning the RMSF should extend by 1/2 on each side in phi-space
    if double:
        nPhi = phiArr_radm2.shape[0]
        nExt = np.ceil(nPhi/2.0)
        resampIndxArr = np.arange(2.0 * nExt + nPhi) - nExt
        phi2Arr = extrap(resampIndxArr, np.arange(nPhi, dtype='int'),
                         phiArr_radm2)
    else:
        phi2Arr = phiArr_radm2

    # Set the weight array
    if weightArr is None:
        weightArr = np.ones(lambdaSqArr_m2.shape, dtype=dtFloat)
    weightArr = np.where(np.isnan(weightArr), 0.0, weightArr)

    # Set the mask array (default to 1D, no masked channels)
    if mskArr is None:
        mskArr = np.zeros_like(lambdaSqArr_m2, dtype="bool")
        nDims = 1
    else:
        mskArr = mskArr.astype("bool")
        nDims = len(mskArr.shape)
    
    # Sanity checks on array sizes
    if not weightArr.shape  == lambdaSqArr_m2.shape:
        log("Err: wavelength^2 and weight arrays must be the same shape.")
        return None, None, None, None
    if not nDims <= 3:
        log("Err: mask dimensions must be <= 3.")
        return None, None, None, None
    if not mskArr.shape[0] == lambdaSqArr_m2.shape[0]:
        log("Err: mask depth does not match lambda^2 vector (%d vs %d).", end=' ')
        (mskArr.shape[0], lambdaSqArr_m2.shape[-1])
        log("     Check that the mask is in [z, y, x] order.")
        return None, None, None, None
    
    # Reshape the mask array to 3 dimensions
    if nDims==1:
        mskArr = np.reshape(mskArr, (mskArr.shape[0], 1, 1))
    elif nDims==2:
        mskArr = np.reshape(mskArr, (mskArr.shape[0], mskArr.shape[1], 1))
    
    # Create a unit cube for use in RMSF calculation (negative of mask)
    #CVE: unit cube removed: it wasn't accurate for non-uniform weights, and was no longer used
    
    # Initialise the complex RM Spread Function cube
    nX = mskArr.shape[-1]
    nY = mskArr.shape[-2]
    nPix = nX * nY
    nPhi = phi2Arr.shape[0]
    RMSFcube = np.ones((nPhi, nY, nX), dtype=dtComplex)

    # If full planes are flagged then set corresponding weights to zero
    xySum =  np.sum(np.sum(mskArr, axis=1), axis=1)
    mskPlanes = np.where(xySum==nPix, 0, 1)
    weightArr *= mskPlanes
    
    # Check for isolated clumps of flags (# flags in a plane not 0 or nPix)
    flagTotals = np.unique(xySum).tolist()
    try:
        flagTotals.remove(0)
    except Exception:
        pass
    try:
        flagTotals.remove(nPix)
    except Exception:
        pass
    do1Dcalc = True
    if len(flagTotals)>0:
        do1Dcalc = False
    
    # lam0Sq is the weighted mean of LambdaSq distribution (B&dB Eqn. 32)
    # Calculate a single lam0Sq_m2 value, ignoring isolated flagged voxels
    K = 1.0 / np.nansum(weightArr)
    lam0Sq_m2 = K * np.nansum(weightArr * lambdaSqArr_m2)

    # Calculate the analytical FWHM width of the main lobe    
    fwhmRMSF = 3.8/(np.nanmax(lambdaSqArr_m2) -
                                  np.nanmin(lambdaSqArr_m2))

    # Do a simple 1D calculation and replicate along X & Y axes
    if do1Dcalc:
        if verbose:
            log("Calculating 1D RMSF and replicating along X & Y axes.")

        # Calculate the RMSF
        a = (-2.0 * 1j * phi2Arr).astype(dtComplex)
        b = (lambdaSqArr_m2 - lam0Sq_m2)
        RMSFArr = K * np.sum(weightArr * np.exp( np.outer(a, b) ), 1)
        
        # Fit the RMSF main lobe
        fitStatus = -1
        if fitRMSF:
            if verbose:
                log("Fitting Gaussian to the main lobe.")
            if fitRMSFreal:
                mp = fit_rmsf(phi2Arr, RMSFArr.real)
            else:
                mp = fit_rmsf(phi2Arr, np.abs(RMSFArr))
            if mp is None or mp.status<1:
                 pass
                 log("Err: failed to fit the RMSF.")
                 log("     Defaulting to analytical value.")
            else:
                fwhmRMSF = mp.params[2]
                fitStatus = mp.status

        # Replicate along X and Y axes
        RMSFcube = np.tile(RMSFArr[:, np.newaxis, np.newaxis], (1, nY, nX))
        fwhmRMSFArr = np.ones((nY,nX), dtype=dtFloat) * fwhmRMSF
        statArr = np.ones((nY, nX), dtype="int") * fitStatus

    # Calculate the RMSF at each pixel
    else:
        if verbose:
            log("Calculating RMSF by channel.")

        # The K value used to scale each RMSF must take into account
        # isolated flagged voxels data in the datacube
        weightCube =  np.invert(mskArr) * weightArr[:, np.newaxis, np.newaxis]
        with np.errstate(divide='ignore', invalid='ignore'):
            KArr = np.true_divide(1.0, np.sum(weightCube, axis=0))
            KArr[KArr == np.inf] = 0
            KArr = np.nan_to_num(KArr)

        # Calculate the RMSF for each plane
        if verbose:
            progress(40, 0)
        a = (lambdaSqArr_m2 - lam0Sq_m2)
        RMSFcube = _calc_rmsf(nPhi, phi2Arr, RMSFcube, KArr, weightCube, a)
        # Default to the analytical RMSF
        fwhmRMSFArr = np.ones((nY, nX), dtype=dtFloat) * fwhmRMSF
        statArr = np.ones((nY, nX), dtype="int") * (-1)
    
        # Fit the RMSF main lobe
        if fitRMSF:
            if verbose:
                log("Fitting main lobe in each RMSF spectrum.")
                log("> This may take some time!")
                progress(40, 0)
            k = 0
            for i in range(nX):
                for j in range(nY):
                    k += 1
                    if verbose:
                        progress(40, ((i+1)*100.0/nPhi))
                    if fitRMSFreal:
                        mp = fit_rmsf(phi2Arr, RMSFcube[:,j,i].real)
                    else:
                        mp = fit_rmsf(phi2Arr, np.abs(RMSFcube[:, j, i]))
                    if not (mp is None or mp.status < 1):
                        fwhmRMSFArr[j, i] = mp.params[2]
                        statArr[j, i] = mp.status

    # Remove redundant dimensions
    RMSFcube = np.squeeze(RMSFcube)
    fwhmRMSFArr = np.squeeze(fwhmRMSFArr)
    statArr = np.squeeze(statArr)
    
    return RMSFcube, phi2Arr, fwhmRMSFArr, statArr

In [12]:
%%timeit
RMSFcube, phi2Arr, fwhmRMSFArr, statArr = get_rmsf_planes(
    lambdaSqArr_m2=lsq_racs,
    phiArr_radm2=phis,
)

192 ms ± 22.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
# %%time
mask = np.zeros((len(lsq_racs), 100, 100)).astype(bool)
mask[:, 50, 50] = True
RMSFcube, phi2Arr, fwhmRMSFArr, statArr = get_rmsf_planes_fast(
    lambdaSqArr_m2=lsq_racs,
    phiArr_radm2=phis,
    mskArr=mask,
    verbose=False,
    fitRMSF=True,
)

(288, 100, 100)